# **Framework, Library, and API Key**

In [1]:
import pandas as pd
import json
import time
import resource 

from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_qdrant import Qdrant

import qdrant_client
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance

from sentence_transformers import CrossEncoder


/workspaces/dev/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **Dataset**

In [2]:
# Load FAQ json.
with open("../data/full_faq.json", "r", encoding="utf-8") as file:
    faq_data = json.load(file)

faq_data[0]


{'question': 'Di mana ada lokasi Rumah Sakit Siloam?',
 'answer': 'Ada 40 Rumah Sakit modern yang terdiri dari 13 Rumah Sakit di Jabodetabek dan 27 rumah sakit yang tersebar di Jawa, Sumatera, Kalimantan, Sulawesi, serta Bali dan Nusa Tenggara.',
 'category': 'FAQ Website'}

# **Embedding Model**

# **intfloat/multilingual-e5-large**

In [3]:
# Initiate FastEmbedEmbeddings
embeddings = FastEmbedEmbeddings(
    cache_dir="../embedding_cache",
    model_name="intfloat/multilingual-e5-large"
)

# Initialize GTE multilingual reranker
reranker = CrossEncoder('Alibaba-NLP/gte-multilingual-reranker-base', trust_remote_code=True)

url = "http://localhost:6333"
collection_name = "faq-question"
distance = Distance.COSINE
dimension = 1024


/workspaces/dev/venv/lib/python3.12/site-packages/langchain_community/embeddings/fastembed.py:109: UserWarning: The model intfloat/multilingual-e5-large now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  values["model"] = fastembed.TextEmbedding(
2025-03-18 23:07:39.885 | ERROR    | fastembed.common.model_management:download_model:429 - Could not download model from HuggingFace: 403 Forbidden: None.
Cannot access content at: https://huggingface.co/api/models/qdrant/multilingual-e5-large-onnx.
Make sure your token has the correct permissions. Falling back to other sources.


100%|██████████| 1.31G/1.31G [00:58<00:00, 22.3MiB/s] 


In [4]:
# Initiate client.
client = QdrantClient("localhost", port=6333)

# Show collections.
collections = client.get_collections()
print(collections)


collections=[CollectionDescription(name='faq-question'), CollectionDescription(name='sales-item')]


In [ ]:
# # Show collection details.
# collection_info = client.get_collection(collection_name="test_faq_openai")
# print(collection_info)


In [ ]:
# # Delete collection.
# client.delete_collection(collection_name="faq-question")
# client.delete_collection(collection_name="faq-question-answer")
# client.delete_collection(collection_name="faq-query-passage")


True

In [6]:
def moveEmbedding(faq_data, batch_size=100):
    """
    Load FAQ data from a JSON-like list of dictionaries and upsert embeddings into Qdrant,
    processing the data in batches.
    
    Each FAQ should have the following keys: 'question', 'answer', and 'category'.
    The text for embedding is constructed by concatenating these fields with a newline.
    """
    client = qdrant_client.QdrantClient(url=url)
    
    # Create collection if it doesn't exist.
    if not client.collection_exists(collection_name=collection_name):
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=dimension, distance=distance),
        )
    
    total_batches = (len(faq_data) - 1) // batch_size + 1
    overall_start_time = time.time()
    
    for batch_num in range(total_batches):
        batch_start_time = time.time()
        start = batch_num * batch_size
        end = start + batch_size
        batch_faq = faq_data[start:end]
        
        texts = []
        ids = []
        payloads = []
        
        for i, faq in enumerate(batch_faq, start=start):
            # Concatenate question and answer with a newline delimiter.
            text = faq['question']
            texts.append(text)
            ids.append(i)
            payloads.append({
                "page_content": text,
                "metadata": {
                    "question": faq['question'],
                    "answer": faq['answer'],
                    "category": faq['category']
                }
            })
        
        # Batch embed the texts.
        batch_embeddings = embeddings.embed_documents(texts)
        
        points = []
        for j, emb in enumerate(batch_embeddings):
            points.append(
                PointStruct(
                    id=ids[j],
                    vector=emb,
                    payload=payloads[j]
                )
            )
        
        # Upsert the current batch of points into Qdrant.
        client.upsert(
            collection_name=collection_name,
            points=points
        )
        
        batch_end_time = time.time()
        batch_elapsed = batch_end_time - batch_start_time
        print(f"Processed batch {batch_num+1}/{total_batches} in {batch_elapsed:.2f} seconds")
    
    overall_end_time = time.time()
    total_elapsed = overall_end_time - overall_start_time

    # Calculate peak memory usage (in MB).
    peak_memory = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024
    print(f"Added: {len(faq_data)} FAQs in {total_elapsed:.2f} seconds, Peak Memory: {peak_memory:.2f} MB")


In [7]:
moveEmbedding(faq_data)

Processed batch 1/4 in 16.25 seconds
Processed batch 2/4 in 8.73 seconds
Processed batch 3/4 in 7.71 seconds
Processed batch 4/4 in 3.60 seconds
Added: 349 FAQs in 36.29 seconds, Peak Memory: 4004.90 MB


In [8]:
client = QdrantClient(
    url=url,
)
qdrant = Qdrant(
    client,
    embeddings=embeddings,
    collection_name=collection_name,
    distance_strategy=distance,
)


/tmp/ipykernel_2415/2148830083.py:4: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.1.2 and will be removed in 0.5.0. Use :class:`~QdrantVectorStore` instead.
  qdrant = Qdrant(


In [9]:
doc = qdrant.similarity_search_with_score('Di mana ada lokasi Rumah Sakit Siloam', score_threshold=0.1, k=20)
for document, score in doc:
    print(f"{score:.4f} | {document.metadata['question']}")


0.9801 | Di mana ada lokasi Rumah Sakit Siloam?
0.9303 | Alamat Siloam Hospitals ASRI dimana?
0.9284 | Rumah sakit Siloam di Jakarta apa saja?
0.9253 | Rumah sakit Siloam di Jabodetabek apa saja?
0.9250 | Alamat Siloam Hospitals Medan dimana?
0.9240 | Alamat Siloam Hospitals Surabaya dimana?
0.9219 | Alamat Siloam Hospitals Jambi dimana?
0.9205 | Alamat Siloam Hospitals Mampang dimana?
0.9205 | Alamat Siloam Hospitals Bogor dimana?
0.9201 | Alamat Siloam Hospitals Kebon Jeruk dimana?
0.9199 | Rumah sakit Siloam di pulau Jawa apa saja?
0.9188 | Alamat Siloam Hospitals Makassar dimana?
0.9181 | Alamat Siloam Hospitals Sentosa dimana?
0.9170 | Alamat Siloam Hospitals Semarang dimana?
0.9157 | Rumah sakit Siloam di pulau Sumatera apa saja?
0.9155 | Alamat Siloam Hospitals Lippo Village dimana?
0.9141 | Rumah sakit Siloam di pulau Kalimantan apa saja?
0.9140 | Rumah sakit Siloam di Tangerang apa saja?
0.9136 | Alamat Siloam Hospitals Banjarmasin dimana?
0.9132 | Alamat Siloam Hospitals Yogy

In [10]:
test_list = [{'query': x['question'], 'expected': x['question']} for x in faq_data]
print(test_list[0])

{'query': 'Di mana ada lokasi Rumah Sakit Siloam?', 'expected': 'Di mana ada lokasi Rumah Sakit Siloam?'}


In [11]:
test_list = test_list + [
    {
        "query": "siloam ada dimana saja",
        "expected": "Di mana ada lokasi Rumah Sakit Siloam?"
    },
    {
        "query": "jam buka mcu",
        "expected": "Jam buka MCU / Medical Check Up?"
    },
    {
        "query": "layanan kesehatan di rumah",
        "expected": "Apa itu Siloam at Home? Apakah Siloam memberikan layanan kesehatan di rumah atau Homecare?"
    },
    {
        "query": "cek obat beli di siloam dari aplikasi",
        "expected": "Halo saya mau cek obat yang saya beli dari Siloam, bisakah dari aplikasi MySiloam?"
    },
    {
        "query": "asuransi kerjasa sama dengan siloam",
        "expected": "Asuransi apa saja yang bekerja sama dengan Siloam?"
    },
    {
        "query": "test",
        "expected": "test"
    },
    
]


In [12]:
def testing(qdrant, q, expected, k=20):
    doc = qdrant.similarity_search_with_score(q, score_threshold=0.1, k=k)
    i = 0
    for d in doc:
        if(d[0].metadata["question"] == expected):
            return [ True, d]
        i += 1
    if len(doc) == 0:
        return [False, "No result"]
    return [False, doc[0]]


In [13]:
# With question only.
results = []
for t in test_list:
    [status, data] = testing(qdrant, t["query"], t["expected"], k=20)
    if(status == False):
        _data = {
            "query": t["query"],
            "result": data[0].metadata['question'] if data != "No result" else "No result",
            "score": data[1] if data != "No result" else "No result"
        }
        results.append(_data)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    df = pd.DataFrame(data=results)
    display(df)
    

,query,result,score
0,test,Bagaimana jika ada rujukan untuk pemeriksaan L...,0.784677


In [14]:
# With question only.
results = []
for t in test_list:
    [status, data] = testing(qdrant, t["query"], t["expected"], k=3)
    if(status == False):
        _data = {
            "query": t["query"],
            "result": data[0].metadata['question'] if data != "No result" else "No result",
            "score": data[1] if data != "No result" else "No result"
        }
        results.append(_data)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    df = pd.DataFrame(data=results)
    display(df)
    

,query,result,score
0,test,Bagaimana jika ada rujukan untuk pemeriksaan L...,0.784677


In [15]:
def test_reranker(test_queries, k=20):
    """
    For each test query in test_queries (each a dict with 'query' and 'expected'),
    perform a two-stage search:
      1. Retrieve candidates using embedding similarity.
      2. Rerank the candidates using the cross-encoder reranker.
    """
    total = len(test_queries)
    orig_top_correct = 0
    rerank_top_correct = 0

    for test in test_queries:
        query = test["query"]
        expected = test["expected"]

        # Stage 1: Get initial candidate results using embedding similarity.
        initial_results = qdrant.similarity_search_with_score(query, score_threshold=0.1, k=k)
        original_top = initial_results[0][0].metadata["question"] if initial_results else None

        # Stage 2: Rerank candidates using the cross-encoder.
        if initial_results:
            rerank_pairs = [[query, doc.metadata["question"]] for doc, _ in initial_results]
            scores = reranker.predict(rerank_pairs)
            # Combine candidates with their rerank scores and sort descending.
            reranked = sorted(zip(initial_results, scores), key=lambda x: x[1], reverse=True)
            reranked_top = reranked[0][0][0].metadata["question"]
        else:
            reranked_top = None

        if original_top == expected:
            orig_top_correct += 1
        if reranked_top == expected:
            rerank_top_correct += 1

        # Minimal output for this query.
        print(f"Query: {query}")
        print(f"Expected: {expected}")
        print(f"Original Top: {original_top}")
        print(f"Reranked Top: {reranked_top}")
        print("-" * 40)

    # Summary of results.
    print("Summary:")
    print(f"Total Queries: {total}")
    print(f"Original Top Correct: {orig_top_correct}")
    print(f"Reranked Top Correct: {rerank_top_correct}")



In [16]:
new_test_list = [
    { "query": "siloam ada dimana saja", "expected": "Di mana ada lokasi Rumah Sakit Siloam?" },
    { "query": "jam buka mcu", "expected": "Jam buka MCU / Medical Check Up?" },
    { "query": "layanan kesehatan di rumah", "expected": "Apa itu Siloam at Home? Apakah Siloam memberikan layanan kesehatan di rumah atau Homecare?" },
    { "query": "cek obat beli di siloam dari aplikasi", "expected": "Halo saya mau cek obat yang saya beli dari Siloam, bisakah dari aplikasi MySiloam?" },
    { "query": "asuransi kerjasa sama dengan siloam", "expected": "Asuransi apa saja yang bekerja sama dengan Siloam?" },
    { "query": "jenis kamar rawat inap apa saja", "expected": "Apa jenis kamar rawat inap yang tersedia?" },
    { "query": "standar keamanan darah di rumah sakit seperti apa", "expected": "Apa standar keamanan suplai darah yang diambil di rumah sakit?" },
    { "query": "fasilitas pusat keunggulan rumah sakit apa saja", "expected": "Fasilitas apa yang dimiliki pusat keunggulan Rumah Sakit?" },
    { "query": "standar kualitas yang diterapkan siloam hospitals", "expected": "Apa standar kualitas yang dipatuhi oleh Siloam Hospitals?" },
    { "query": "pembayaran apa saja tersedia di siloam hospitals", "expected": "Metode pembayaran apa yang tersedia di Siloam Hospitals?" },
    { "query": "kamar rumah sakit menyediakan fasilitas apa saja", "expected": "Fasilitas apa yang disediakan di dalam kamar?" },
    { "query": "apakah merokok diperbolehkan di rumah sakit", "expected": "Apakah merokok diperbolehkan di area rumah sakit?" },
    { "query": "berapa banyak pasien di siloam hospitals tiap tahun", "expected": "Berapa banyak pasien yang dirawat Siloam Hospitals setiap tahun?" },
    { "query": "apakah siloam hospitals memberikan perkiraan biaya perawatan", "expected": "Apakah rumah sakit Siloam menawarkan perkiraan biaya perawatan?" },
    { "query": "asuransi kesehatan diterima di siloam hospitals", "expected": "Jenis asuransi kesehatan apa saja yang diterima di Siloam Hospitals?" },
    { "query": "bagaimana jika dokter pilihan tidak ada", "expected": "Bagaimana jika dokter pilihan saya tidak tersedia?" },
    { "query": "cara berkomunikasi dengan dokter bagi yang tidak fasih bahasa indonesia", "expected": "Saya tidak fasih berbahasa Indonesia. Bagaimana cara berkomunikasi dengan dokter?" },
    { "query": "barang apa yang harus dibawa ke siloam hospital", "expected": "Barang apa saja yang perlu saya bawa ke Siloam Hospital?" },
    { "query": "apakah perlu janji untuk spesialis", "expected": "Apakah saya perlu membuat janji untuk bertemu dengan spesialis?" },
    { "query": "jam kunjungan pasien di rumah sakit", "expected": "Pada jam berapa pengunjung diperbolehkan untuk mengunjungi pasien?" },
    { "query": "bolehkah kasih bunga atau hadiah ke pasien", "expected": "Apakah saya boleh memberikan bunga atau hadiah kepada pasien di Siloam Hospital?" },
    { "query": "cara hubungi pasien di ICU", "expected": "Bagaimana saya bisa menghubungi pasien di Unit Perawatan Intensif?" },
    { "query": "ada daftar hotel di sekitar siloam hospitals", "expected": "Apakah Anda memiliki daftar hotel dekat Rumah Sakit Siloam?" },
    { "query": "tamu boleh kunjungan setelah jam kerja tidak", "expected": "Apakah tamu diizinkan untuk berkunjung setelah jam kerja?" },
    { "query": "anak-anak boleh kunjungi pasien di rumah sakit", "expected": "Apakah anak-anak diperbolehkan mengunjungi pasien di Siloam Hospitals?" },
    { "query": "dokumen apa yang harus dibawa untuk ambil medical report", "expected": "Dokumen apa saja yang harus saya bawa saat ingin mengambil medical report/formulir asuransi/dokumen medis?" },
    { "query": "cara minta informasi medis", "expected": "Bagaimana cara saya meminta informasi medis saya?" },
    { "query": "cara dapat salinan rekam medis", "expected": "Bagaimana saya bisa mendapatkan salinan rekam medis saya untuk diri saya sendiri atau untuk dikirimkan kepada orang lain selain diri saya sendiri, seperti dokter, pengacara, perusahaan asuransi atau orang lain?" },
    { "query": "jam operasional mcu", "expected": "Jam buka MCU / Medical Check Up?" },
    { "query": "apakah perlu puasa sebelum mcu", "expected": "Apakah MCU / Medical Check Up perlu melakukan puasa?" },
    { "query": "berapa lama waktu untuk mcu test", "expected": "Berapa lama waktu yang dibutuhkan untuk MCU / Medical Check Up tes?" },
    { "query": "paket mcu rekomendasi apa yang terlengkap", "expected": "Apa rekomendasi paket MCU / Medical Check Up terlengkap saat ini?" },
    { "query": "apakah semua siloam hospitals sediakan mcu", "expected": "Apakah saya bisa melakukan MCU / Medical Check Up di semua Siloam?" },
    { "query": "cara daftar mcu", "expected": "Bagaimana cara mendaftar MCU / Medical Check Up?" },
    { "query": "apa yang harus dibawa untuk mcu", "expected": "Apa yang perlu dibawa ketika melakukan MCU / Medical Check Up?" },
    { "query": "bedanya daftar mcu lewat website dan telepon", "expected": "Apa bedanya mendaftar MCU / Medical Check Up lewat website dengan lewat telepon/datang langsung?" },
    { "query": "metode pembayaran untuk mcu", "expected": "Apa saja metode pembayaran pada pendaftaran MCU / Medical Check Up?" },
    { "query": "hasil mcu yang didapatkan", "expected": "Hasil apa yang saya dapatkan setelah tes MCU / Medical Check Up di Siloam?" },
    { "query": "berapa lama hasil mcu keluar", "expected": "Berapa lama hasil dari MCU / Medical Check Up tes?" },
    { "query": "persiapan sebelum datang ke mcu", "expected": "Apa saja yang harus saya lakukan sebelum datang ke MCU / Medical Check Up (persiapan yang perlu saya lakukan)?" },
    { "query": "siloam at home itu apa", "expected": "Apa itu Siloam at Home? Apakah Siloam memberikan layanan kesehatan di rumah atau Homecare?" },
    { "query": "layanan siloam at home apa saja", "expected": "Layanan Siloam at Home apa saja yang diberikan oleh Siloam?" },
    { "query": "cara pesan layanan siloam at home", "expected": "Bagaimana cara melakukan pemesanan layanan Siloam at Home?" },
    { "query": "metode pembayaran untuk siloam at home", "expected": "Apa saja metode pembayaran yang saya bisa gunakan untuk Siloam at Home?" },
    { "query": "apakah siloam at home sediakan perawat di rumah", "expected": "Apakah Siloam at Home memfasilitasi layanan perawat tinggal di rumah pasien?" },
    { "query": "cek obat dari aplikasi siloam", "expected": "Halo saya mau cek obat yang saya beli dari Siloam, bisakah dari aplikasi MySiloam?" },
    { "query": "cara pesan dokter lewat aplikasi mysiloam", "expected": "Bagaimana cara pesan dokter dari aplikasi MySiloam?" },
    { "query": "jika booking lewat aplikasi, apakah slot sesuai jadwal dokter", "expected": "Jika saya udah booking dari aplikasi MySiloam, apakah saya pasti terdaftar slot yang sesuai dengan jadwal dokter?" },
    { "query": "sudah booking di mysiloam, apakah saya terdaftar", "expected": "Saya sudah booking di aplikasi MySiloam, apakah saya sudah terdaftar?" },
    { "query": "fitur apa saja di aplikasi mysiloam", "expected": "Fitur apa sajakah yang ada di aplikasi MySiloam?" },
    { "query": "cara download aplikasi mysiloam", "expected": "Bagaimana cara download aplikasi MySiloam?" },
    { "query": "apa itu menu rekam medis di mysiloam", "expected": "Apa itu menu rekam medis pada aplikasi MySiloam?" },
    { "query": "cara aktivasi rekam medis", "expected": "Gimana caranya untuk buka atau aktivasi rekam medis?" },
    { "query": "data apa saja ada di menu rekam medis", "expected": "Data apa saja yang tersedia di menu rekam medis di aplikasi MySiloam?" },
    { "query": "data rawat inap yang ditampilkan apa saja", "expected": "Data apa saja yang tersedia untuk rawat inap?" },
    { "query": "asuransi yang kerjasama dengan siloam apa saja", "expected": "Asuransi apa saja yang bekerja sama dengan Siloam?" },
    { "query": "jika asuransi tidak kerjasama, apa yang harus dilakukan", "expected": "Apa yang harus dilakukan jika asuransi tidak bekerja sama dengan rumah sakit?" },
    { "query": "cara bayar dengan asuransi", "expected": "Bagaimana alur dalam membayar dengan asuransi?" },
    { "query": "berapa lama urusan asuransi sebelum konsultasi dokter", "expected": "Berapa lama waktu untuk pasien untuk mengurus asuransi sebelum konsultasi dengan dokter?" },
    { "query": "apa yang harus disiapkan saat daftar dengan asuransi", "expected": "Apa saja yang harus dipersiapkan saat mendaftar ke rumah sakit menggunakan asuransi?" },
    { "query": "data apa yang diperlukan saat pakai bpjs", "expected": "Data apa yang dibutuhkan saat menggunakan BPJS di Siloam Hospitals?" },
    { "query": "apakah kondisi darurat perlu surat rujukan", "expected": "Apakah kondisi gawat darurat memerlukan Surat Rujukan dari Puskesmas/Klinik seperti mendaftar rawat jalan?" },
    { "query": "rumah sakit siloam mana aja terima bpjs", "expected": "Rumah Sakit Siloam mana saja yang menerima pasien BPJS?" },
    { "query": "bisakah daftar bpjs di hari h", "expected": "Apakah saya bisa daftar BPJS di hari H?" },
    { "query": "jam operasional konsultasi bpjs", "expected": "Jam operasional untuk konsultasi BPJS?" },
    { "query": "cara reschedule appointment bpjs", "expected": "Bagaimana cara melakukan reschedule appointment apabila saya sudah melakukan pendaftaran via website?" },
    { "query": "masa berlaku surat rujukan bpjs", "expected": "Berapa lama masa berlaku Surat Rujukan yang diterbitkan Puskesmas/Klinik untuk berobat ke Rumah Sakit?" },
    { "query": "cara buat appointment bpjs", "expected": "Bagaimana cara membuat appointment/ mendaftar untuk antrian untuk BPJS?" },
    { "query": "cara dapatkan surat rujukan siloam", "expected": "Bagaimana cara mendapatkan surat rujukan untuk diperiksa oleh RS Siloam?" },
    { "query": "dimana beli produk laboratorium online", "expected": "Dimana saya dapat membeli produk laboratorium / radiologi secara online?" },
    { "query": "produk yang dicari tidak ada di website", "expected": "Produk yang saya cari tidak ada di website" },
    { "query": "cek ketersediaan prosedur di rumah sakit", "expected": "Ketersediaan prosedur pada rumah sakit" },
    { "query": "perlukah surat rujukan dokter untuk pembelian", "expected": "Apakah memerlukan surat rujukan dari dokter?" },
    { "query": "apakah jadwal di website sudah sesuai", "expected": "Apakah jadwal yang ditampilkan sudah sesuai?" },
    { "query": "biaya sudah termasuk konsultasi atau tidak", "expected": "Apakah biaya yang dibayarkan sudah termasuk dengan biaya konsultasi?" },
    { "query": "cara atur ulang jadwal", "expected": "Bagaimana jika saya ingin mengatur jadwal ulang?" },
    { "query": "ada batas atur ulang jadwal?", "expected": "Apakah ada limitasi untuk mengatur jadwal ulang?" },
    { "query": "jika butuh layanan homecare, bagaimana caranya", "expected": "Bagaimana jika saya butuh layanan homecare?" },
    { "query": "apakah bisa pesan homeservice dari semua siloam hospitals", "expected": "Apakah saya bisa melakukan pemesanan homeservice dari semua rumah sakit Siloam?" },
    { "query": "cara minta pengembalian dana", "expected": "Bagaimana jika saya ingin pengembalian dana?" },
    { "query": "pendaftaran radiologi di hari h boleh gak", "expected": "Apakah pendaftaran radiologi dapat dilakukan pada hari H?" },
    { "query": "jam operasional radiologi emergency", "expected": "Jam operasional radiologi?" },
    { "query": "jam operasional laboratorium untuk non-emergency", "expected": "Jam operasional laboratorium?" },
    { "query": "lokasi radiologi atau laboratorium di rumah sakit", "expected": "Dimanakah lokasi radiologi/laboratorium?" },
    { "query": "cara pembayaran untuk laboratorium", "expected": "Cara pembayaran?" },
    { "query": "cara pemesanan laboratorium", "expected": "Cara pemesanan?" },
    { "query": "bagaimana memasukkan diskon di pemesanan laboratorium", "expected": "Bagaimana cara memasukan diskon pada pemesanan laboratorium dan radiologi di Website?" },
    { "query": "cara pesan telekonsultasi", "expected": "Bagaimana cara melakukan pemesanan Telekonsultasi?" },
    { "query": "prosedur telekonsultasi seperti apa", "expected": "Bagaimana prosedur melakukan Telekonsultasi?" },
    { "query": "setelah pesan telekonsultasi, apa yang didapat", "expected": "Apa yang akan didapatkan setelah melakukan pemesanan Telekonsultasi?" },
    { "query": "dimana dapatkan link telekonsultasi", "expected": "Dimana saya bisa mendapatkan link Telekonsultasi?" },
    { "query": "berapa lama durasi telekonsultasi", "expected": "Berapa lama durasi Telekonsultasi?" },
    { "query": "dokter bisa apa saja di telekonsultasi", "expected": "Apa saja yang Dokter bisa lakukan dalam Teleconsultasi?" },
    { "query": "jika salah pesan telekonsultasi, bagaimana caranya", "expected": "Bagaimana jika saya salah dalam melakukan pemesanan telekonsultasi?" },
    { "query": "prosedur refund telekonsultasi", "expected": "Bagaimana prosedur untuk pengembalian dana yang sudah saya bayarkan pada pemesanan Telekonsultasi?" },
    { "query": "bisa atur ulang jadwal telekonsultasi di hari h gak", "expected": "Apakah bisa melakukan atur ulang jadwal Telekonsultasi di hari H?" },
    { "query": "jika dokter tidak hadir di telekonsultasi, apa yang dilakukan", "expected": "Bagaimana jika Dokter berhalangan untuk hadir ketika Teleconsultation saya?" },
    { "query": "setelah telekonsultasi, bagaimana alur resep obat", "expected": "Bagaimana alur jika saya di resepkan obat oleh Dokter setelah selesai melakukan Telekonsultasi?" },
    { "query": "jika ada rujukan untuk lab dan radiology, apa yang harus dilakukan", "expected": "Bagaimana jika ada rujukan untuk pemeriksaan Lab dan Radiology?" },
    { "query": "cara pesan telechat", "expected": "Bagaimana cara melakukan pemesanan Telechat?" }
]


In [17]:
# Run the testing function.
test_reranker(new_test_list)

Query: siloam ada dimana saja
Expected: Di mana ada lokasi Rumah Sakit Siloam?
Original Top: Di mana ada lokasi Rumah Sakit Siloam?
Reranked Top: Alamat SILOAM CLINIC SOE dimana?
----------------------------------------
Query: jam buka mcu
Expected: Jam buka MCU / Medical Check Up?
Original Top: Jam buka MCU / Medical Check Up?
Reranked Top: Jam buka MCU / Medical Check Up?
----------------------------------------
Query: layanan kesehatan di rumah
Expected: Apa itu Siloam at Home? Apakah Siloam memberikan layanan kesehatan di rumah atau Homecare?
Original Top: Ketersediaan prosedur pada rumah sakit
Reranked Top: Bagaimana jika saya butuh layanan homecare?
----------------------------------------
Query: cek obat beli di siloam dari aplikasi
Expected: Halo saya mau cek obat yang saya beli dari Siloam, bisakah dari aplikasi MySiloam?
Original Top: Halo saya mau cek obat yang saya beli dari Siloam, bisakah dari aplikasi MySiloam?
Reranked Top: Halo saya mau cek obat yang saya beli dari Sil

In [18]:
def compare_query_speed(test_queries, k=20):
    """
    Membandingkan kecepatan eksekusi query tanpa reranker vs dengan reranker.
    
    Untuk setiap query pada test_queries (setiap elemen merupakan dict dengan key 'query'),
    fungsi ini melakukan:
      1. Pencarian awal menggunakan similarity embedding (tanpa reranker).
      2. Pencarian lengkap dengan reranker cross-encoder.
    
    Waktu eksekusi untuk masing-masing metode diukur dan dicetak per query,
    serta dirangkum sebagai rata-rata untuk seluruh query.
    """
    total_queries = len(test_queries)
    total_time_no_rerank = 0.0
    total_time_rerank = 0.0

    for test in test_queries:
        query = test["query"]

        # Ukur waktu untuk pencarian tanpa reranker.
        start_time = time.time()
        initial_results = qdrant.similarity_search_with_score(query, score_threshold=0.1, k=k)
        time_no_rerank = time.time() - start_time
        total_time_no_rerank += time_no_rerank

        # Ukur waktu untuk pencarian dengan reranker.
        start_time = time.time()
        if initial_results:
            rerank_pairs = [[query, doc.metadata["question"]] for doc, _ in initial_results]
            scores = reranker.predict(rerank_pairs)
            # Reranking: mengurutkan hasil berdasarkan skor reranker secara menurun.
            reranked = sorted(zip(initial_results, scores), key=lambda x: x[1], reverse=True)
        time_rerank = time.time() - start_time
        total_time_rerank += time_rerank

        print(f"Query: {query}")
        print(f"Waktu tanpa reranker: {time_no_rerank:.4f} detik")
        print(f"Waktu dengan reranker: {time_rerank:.4f} detik")
        print("-" * 40)

    avg_time_no_rerank = total_time_no_rerank / total_queries if total_queries > 0 else 0
    avg_time_rerank = total_time_rerank / total_queries if total_queries > 0 else 0

    print("Ringkasan:")
    print(f"Total Query: {total_queries}")
    print(f"Rata-rata waktu tanpa reranker: {avg_time_no_rerank:.4f} detik")
    print(f"Rata-rata waktu dengan reranker: {avg_time_rerank:.4f} detik")


In [19]:
compare_query_speed(new_test_list)

Query: siloam ada dimana saja
Waktu tanpa reranker: 0.0580 detik
Waktu dengan reranker: 0.8030 detik
----------------------------------------
Query: jam buka mcu
Waktu tanpa reranker: 0.0476 detik
Waktu dengan reranker: 1.0129 detik
----------------------------------------
Query: layanan kesehatan di rumah
Waktu tanpa reranker: 0.0512 detik
Waktu dengan reranker: 1.1290 detik
----------------------------------------
Query: cek obat beli di siloam dari aplikasi
Waktu tanpa reranker: 0.0616 detik
Waktu dengan reranker: 1.0705 detik
----------------------------------------
Query: asuransi kerjasa sama dengan siloam
Waktu tanpa reranker: 0.0693 detik
Waktu dengan reranker: 0.8705 detik
----------------------------------------
Query: jenis kamar rawat inap apa saja
Waktu tanpa reranker: 0.0612 detik
Waktu dengan reranker: 1.0918 detik
----------------------------------------
Query: standar keamanan darah di rumah sakit seperti apa
Waktu tanpa reranker: 0.0621 detik
Waktu dengan reranker: 1.

In [ ]:
from fastembed import SparseTextEmbedding

# Create the DataFrame and clean it up
supported_models = (
    pd.DataFrame(SparseTextEmbedding.list_supported_models())
    .sort_values("size_in_GB")
    .drop(columns=["sources", "model_file", "additional_files", "license", "requires_idf"])
    .reset_index(drop=True)
)

# Filter model.
# filtered_models = supported_models[
#     supported_models['description'].str.contains("multilingual", case=False, na=False) &
#     ~supported_models['model'].str.contains("jinaai", case=False, na=False)
# ]

# Display the filtered DataFrame with full column content
with pd.option_context('display.max_colwidth', None):
    display(supported_models)


,model,sources,model_file,description,license,size_in_GB,additional_files,requires_idf,vocab_size
0,Qdrant/bm25,"{'hf': 'Qdrant/bm25', 'url': None}",mock.file,BM25 as sparse embeddings meant to be used with Qdrant,apache-2.0,0.010,"[arabic.txt, azerbaijani.txt, basque.txt, bengali.txt, catalan.txt, chinese.txt, danish.txt, dutch.txt, english.txt, finnish.txt, french.txt, german.txt, greek.txt, hebrew.txt, hinglish.txt, hungarian.txt, indonesian.txt, italian.txt, kazakh.txt, nepali.txt, norwegian.txt, portuguese.txt, romanian.txt, russian.txt, slovene.txt, spanish.txt, swedish.txt, tajik.txt, turkish.txt]",True,0
1,Qdrant/bm42-all-minilm-l6-v2-attentions,"{'hf': 'Qdrant/all_miniLM_L6_v2_with_attentions', 'url': None}",model.onnx,"Light sparse embedding model, which assigns an importance score to each token in the text",apache-2.0,0.090,[stopwords.txt],True,30522
2,prithvida/Splade_PP_en_v1,"{'hf': 'Qdrant/SPLADE_PP_en_v1', 'url': None}",model.onnx,Independent Implementation of SPLADE++ Model for English.,apache-2.0,0.532,[],None,30522
3,prithivida/Splade_PP_en_v1,"{'hf': 'Qdrant/SPLADE_PP_en_v1', 'url': None}",model.onnx,Independent Implementation of SPLADE++ Model for English.,apache-2.0,0.532,[],None,30522


In [20]:
from fastembed import SparseTextEmbedding

SparseTextEmbedding.list_supported_models()

[{'model': 'prithivida/Splade_PP_en_v1',
  'sources': {'hf': 'Qdrant/SPLADE_PP_en_v1', 'url': None},
  'model_file': 'model.onnx',
  'description': 'Independent Implementation of SPLADE++ Model for English.',
  'license': 'apache-2.0',
  'size_in_GB': 0.532,
  'additional_files': [],
  'requires_idf': None,
  'vocab_size': 30522},
 {'model': 'prithvida/Splade_PP_en_v1',
  'sources': {'hf': 'Qdrant/SPLADE_PP_en_v1', 'url': None},
  'model_file': 'model.onnx',
  'description': 'Independent Implementation of SPLADE++ Model for English.',
  'license': 'apache-2.0',
  'size_in_GB': 0.532,
  'additional_files': [],
  'requires_idf': None,
  'vocab_size': 30522},
 {'model': 'Qdrant/bm42-all-minilm-l6-v2-attentions',
  'sources': {'hf': 'Qdrant/all_miniLM_L6_v2_with_attentions', 'url': None},
  'model_file': 'model.onnx',
  'description': 'Light sparse embedding model, which assigns an importance score to each token in the text',
  'license': 'apache-2.0',
  'size_in_GB': 0.09,
  'additional_f